In [ ]:
!pip install pandas numpy pyodbc sqlalchemy faker


In [ ]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import date
import pyodbc

# -----------------------------
# 1) Connection details
# -----------------------------
SERVER = "vwuk-sql-server-hithaishree.database.windows.net"
DATABASE = "vwuk_analytics_db"
USERNAME = "sqladmin"
PASSWORD = "Hithu25#"   # <-- your password

conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={USERNAME};"
    f"PWD={PASSWORD};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "Connection Timeout=30;"
)

cn = pyodbc.connect(conn_str)
cn.autocommit = False
cur = cn.cursor()

print("✅ Connected to Azure SQL successfully")


In [ ]:
import pandas as pd
from datetime import date

start = date(2024, 1, 1)
end   = date(2025, 12, 31)

dates = pd.date_range(start=start, end=end, freq="D")

dim_date = pd.DataFrame({
    "DateKey": dates.strftime("%Y%m%d").astype(int),
    "Date": dates.date,
    "Year": dates.year.astype(int),
    "Month": dates.month.astype(int),
    "MonthName": dates.strftime("%b"),
    "Quarter": ((dates.month - 1)//3 + 1).astype(int),
    "WeekOfYear": dates.isocalendar().week.astype(int),
    "DayOfMonth": dates.day.astype(int),
    "DayName": dates.strftime("%a"),
    "IsWeekend": (dates.weekday >= 5).astype(int)
})

cur.fast_executemany = True

cur.executemany(
    """
    INSERT INTO dw.DimDate
    (DateKey,[Date],[Year],[Month],MonthName,[Quarter],WeekOfYear,DayOfMonth,DayName,IsWeekend)
    VALUES (?,?,?,?,?,?,?,?,?,?)
    """,
    list(dim_date.itertuples(index=False, name=None))
)

cn.commit()
print("✅ DimDate loaded:", len(dim_date))


In [ ]:
supplier_types = ["Subcontractor","Materials","Plant Hire","Design"]

suppliers = []
for i in range(1, 19):  # 18 suppliers
    suppliers.append((
        f"SUP-{2000+i}",
        fake.company(),
        np.random.choice(supplier_types),
        "United Kingdom",
        int(np.random.rand() < 0.3)
    ))

cur.executemany(
    """
    INSERT INTO dw.DimSupplier
    (SupplierCode,SupplierName,SupplierType,Country,IsPreferred)
    VALUES (?,?,?,?,?)
    """,
    suppliers
)

cn.commit()
print("✅ DimSupplier loaded:", len(suppliers))


In [ ]:
supplier_types = ["Subcontractor","Materials","Plant Hire","Design"]

suppliers = []
for i in range(1, 19):
    code = f"SUP-{2000+i}"
    suppliers.append((
        code,                      # <-- for IF NOT EXISTS
        code,                      # <-- for INSERT SupplierCode
        fake.company(),
        np.random.choice(supplier_types),
        "United Kingdom",
        int(np.random.rand() < 0.3)
    ))

cur.executemany(
    """
    IF NOT EXISTS (
        SELECT 1
        FROM dw.DimSupplier
        WHERE SupplierCode = ?
          AND IsCurrent = 1
    )
    INSERT INTO dw.DimSupplier
        (SupplierCode,SupplierName,SupplierType,Country,IsPreferred)
    VALUES (?,?,?,?,?)
    """,
    suppliers
)

cn.commit()
print("✅ DimSupplier upsert done (duplicates skipped)")


In [ ]:
import pandas as pd

pd.read_sql("""
SELECT
  (SELECT COUNT(*) FROM dw.DimProject WHERE IsCurrent=1) AS ProjectCnt,
  (SELECT COUNT(*) FROM dw.DimPackage WHERE IsCurrent=1) AS PackageCnt,
  (SELECT COUNT(*) FROM dw.DimSupplier WHERE IsCurrent=1) AS SupplierCnt,
  (SELECT COUNT(*) FROM dw.DimDate) AS DateCnt
""", cn)


In [ ]:
from faker import Faker
import numpy as np

fake = Faker("en_GB")
np.random.seed(42)

regions = ["UK South","UK North","London","Midlands","Scotland"]
bus = ["Civils","Rail","Buildings","Infrastructure"]
contract_types = ["NEC","JCT","Framework"]
statuses = ["Planned","Live","Complete","On Hold"]

projects = []
for i in range(1, 13):  # 12 projects
    code = f"PRJ-{1000+i}"
    projects.append((
        code,  # for IF NOT EXISTS
        code,  # for INSERT ProjectCode
        f"{fake.city()} {np.random.choice(['Station Upgrade','Road Scheme','Bridge Works','School Build','Depot Fit-out'])}",
        np.random.choice(regions),
        np.random.choice(bus),
        np.random.choice(contract_types),
        fake.company(),
        fake.date_between("-2y","-6M"),
        fake.date_between("+3M","+18M"),
        np.random.choice(statuses, p=[0.1,0.6,0.2,0.1]),
        fake.name(),
        f"CC-{np.random.randint(100,999)}"
    ))

cur.executemany(
    """
    IF NOT EXISTS (
        SELECT 1
        FROM dw.DimProject
        WHERE ProjectCode = ?
          AND IsCurrent = 1
    )
    INSERT INTO dw.DimProject
    (ProjectCode,ProjectName,Region,BusinessUnit,ContractType,ClientName,
     StartDate,PlannedEndDate,Status,ProjectManager,CostCentre)
    VALUES (?,?,?,?,?,?,?,?,?,?,?)
    """,
    projects
)

cn.commit()
print("✅ DimProject upsert done (duplicates skipped)")


In [ ]:
packages = [
    ("PKG-01","Groundworks","Civils","High"),
    ("PKG-02","Concrete","Civils","Med"),
    ("PKG-03","Steel","Buildings","High"),
    ("PKG-04","MEP","Buildings","High"),
    ("PKG-05","Paving & Surfacing","Infrastructure","Med"),
    ("PKG-06","Signalling","Rail","High"),
    ("PKG-07","Track Works","Rail","High"),
    ("PKG-08","Design & Survey","Infrastructure","Low"),
]

# repeat package code twice because query uses 1 placeholder + insert uses 4
packages_upsert = [(p[0], p[0], p[1], p[2], p[3]) for p in packages]

cur.executemany(
    """
    IF NOT EXISTS (
        SELECT 1
        FROM dw.DimPackage
        WHERE PackageCode = ?
          AND IsCurrent = 1
    )
    INSERT INTO dw.DimPackage
    (PackageCode,PackageName,Workstream,RiskCategory)
    VALUES (?,?,?,?)
    """,
    packages_upsert
)

cn.commit()
print("✅ DimPackage upsert done (duplicates skipped)")


In [ ]:
pd.read_sql("""
SELECT
  (SELECT COUNT(*) FROM dw.DimProject WHERE IsCurrent=1) AS ProjectCnt,
  (SELECT COUNT(*) FROM dw.DimPackage WHERE IsCurrent=1) AS PackageCnt,
  (SELECT COUNT(*) FROM dw.DimSupplier WHERE IsCurrent=1) AS SupplierCnt,
  (SELECT COUNT(*) FROM dw.DimDate) AS DateCnt
""", cn)


In [ ]:
import pyodbc

SERVER   = "vwuk-sql-server-hithaishree.database.windows.net"
DATABASE = "vwuk_analytics_db"
USERNAME = "sqladmin"
PASSWORD = "Hithu25#"   # <-- same one you used earlier

conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={USERNAME};"
    f"PWD={PASSWORD};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "Connection Timeout=30;"
)

cn = pyodbc.connect(conn_str)
cur = cn.cursor()

print("✅ Connected to Azure SQL")


In [ ]:
import pandas as pd
pd.read_sql("SELECT DB_NAME() AS current_db;", cn)


In [ ]:
import numpy as np
import pandas as pd

# pull keys
proj_keys = pd.read_sql("SELECT ProjectKey FROM dw.DimProject WHERE IsCurrent=1;", cn)["ProjectKey"].tolist()
sup_keys  = pd.read_sql("SELECT SupplierKey FROM dw.DimSupplier WHERE IsCurrent=1;", cn)["SupplierKey"].tolist()
pkg_keys  = pd.read_sql("SELECT PackageKey FROM dw.DimPackage WHERE IsCurrent=1;", cn)["PackageKey"].tolist()
date_keys = pd.read_sql("SELECT DateKey FROM dw.DimDate;", cn)["DateKey"].tolist()

np.random.seed(42)

fact_rows = []
for _ in range(5000):
    dk = int(np.random.choice(date_keys))
    p  = int(np.random.choice(proj_keys))
    s  = int(np.random.choice(sup_keys))
    pk = int(np.random.choice(pkg_keys))

    committed = max(0, np.random.normal(25000, 12000))
    actual    = max(0, committed * np.random.uniform(0.3, 1.1))
    appr_var  = max(0, np.random.normal(1500, 2000))
    pend_var  = max(0, np.random.normal(800, 1200))
    forecast  = max(0, (committed + appr_var) * np.random.uniform(0.9, 1.2))

    load_key = f"{dk}-{p}-{pk}-{s}-{np.random.randint(1, 10_000_000)}"

    # NOTE: load_key is included TWICE (1st for IF NOT EXISTS, 2nd for INSERT LoadKey)
    fact_rows.append((
        load_key,  # for IF NOT EXISTS
        load_key,  # for INSERT column LoadKey
        dk, p, pk, s, "GBP",
        round(committed,2), round(actual,2),
        round(appr_var,2), round(pend_var,2),
        round(forecast,2),
        "Synthetic"
    ))

cur.fast_executemany = True

cur.executemany(
    """
    IF NOT EXISTS (SELECT 1 FROM dw.FactCost WHERE LoadKey = ?)
    INSERT INTO dw.FactCost
    (LoadKey,DateKey,ProjectKey,PackageKey,SupplierKey,CurrencyCode,
     CommittedCost,ActualCost,ApprovedVariation,PendingVariation,ForecastEAC,SourceSystem)
    VALUES (?,?,?,?,?,?,?,?,?,?,?,?)
    """,
    fact_rows
)

cn.commit()
print("✅ FactCost load attempted:", len(fact_rows))


In [ ]:
pd.read_sql("SELECT DB_NAME() AS current_db;", cn)


In [ ]:
pd.read_sql("""
SELECT c.name
FROM sys.columns c
JOIN sys.tables t ON c.object_id = t.object_id
JOIN sys.schemas s ON t.schema_id = s.schema_id
WHERE s.name='dw' AND t.name='FactCost' AND c.name='LoadKey';
""", cn)


In [ ]:
pd.read_sql("SELECT COUNT(*) AS FactRows FROM dw.FactCost;", cn)


In [ ]:
pd.read_sql("""
SELECT TOP 5 LoadKey, DateKey, ProjectKey, PackageKey, SupplierKey, CommittedCost, ActualCost, ForecastEAC
FROM dw.FactCost
ORDER BY 1 DESC;
""", cn)
